In [ ]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 23.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset
import pandas as pd

pd.options.display.max_columns= None

In [ ]:
dataset = load_dataset("AnikaBasu/CyberbullyingDataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2956 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'text', 'instruction', 'output'],
        num_rows: 2956
    })
})

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df.head()

,input,text,instruction,output
0,,Here is a classification that determines wheth...,"In other words #katandandre, your food was cra...",not_cyberbullying
1,,Here is a classification that determines wheth...,"In other words #katandandre, your food was cra...",not_cyberbullying
2,,Here is a classification that determines wheth...,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
3,,Here is a classification that determines wheth...,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
4,,Here is a classification that determines wheth...,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying


In [ ]:
def drop_columns(df, columns):
    return df.drop(columns=columns, inplace=False)

columns_to_drop = ['input', 'text']
df = drop_columns(df, columns_to_drop)


In [ ]:
df.head()

,instruction,output
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,"In other words #katandandre, your food was cra...",not_cyberbullying
2,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
3,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
4,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying


In [ ]:
df['output'].value_counts()

output
gender                 612
religion               562
other_cyberbullying    483
not_cyberbullying      434
age                    433
ethnicity              432
Name: count, dtype: int64

In [ ]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Assign values to cyberbullying types
cyberbullying_type_mapping = {
    "age": 1,
    "ethnicity": 2,
    "gender": 3,
    "not_cyberbullying": 4,
    "other_cyberbullying": 5,
    "religion": 6
}
df['output'] = df['output'].map(cyberbullying_type_mapping)

In [ ]:
df.head()

,instruction,output
0,"In other words #katandandre, your food was cra...",4
1,"In other words #katandandre, your food was cra...",4
2,Why is #aussietv so white? #MKR #theblock #ImA...,4
3,@XochitlSuckkks a classy whore? Or more red ve...,4
4,"@Jason_Gio meh. :P thanks for the heads up, b...",4


In [ ]:
def preprocess_text(text):

    # Lowercasing
    text = text.lower()

    # Replace links with 'link'
    text = re.sub(r'http\S+|www\S+', 'link', text)

    # Remove mentions (words starting with @)
    text = re.sub(r'@\w+', '', text)

    #remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Removing Punctuation and Special Characters
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Applying text preprocessing to the 'instruction' column in the DataFrame
df['instruction'] = df['instruction'].apply(preprocess_text)

print(df.head())

# Function to remove emoticons from text
def remove_emoticons(text):
    return re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', '', text)

df['instruction'] = df['instruction'].apply(remove_emoticons)

                                         instruction  output
0                            words food crapilicious       4
1                            words food crapilicious       4
2                                              white       4
3                   classy whore red velvet cupcakes       4
4  meh p thanks heads concerned another angry dud...       4


In [ ]:

#Tokenize data
from nltk.tokenize import word_tokenize

df['tokens'] = df['instruction'].apply(word_tokenize)

# Display the DataFrame with tokenized text
print(df[['instruction', 'tokens']].head())

                                         instruction  \
0                            words food crapilicious   
1                            words food crapilicious   
2                                              white   
3                   classy whore red velvet cupcakes   
4  meh p thanks heads concerned another angry dud...   

                                              tokens  
0                        [words, food, crapilicious]  
1                        [words, food, crapilicious]  
2                                            [white]  
3             [classy, whore, red, velvet, cupcakes]  
4  [meh, p, thanks, heads, concerned, another, an...  


In [ ]:
# NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


# Function to remove stopwords
def remove_stopwords(text):
    return ' '.join(word for word in text.split() if word.lower() not in stop_words)

df['instruction'] = df['instruction'].apply(remove_stopwords)

df['instruction']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0                                 words food crapilicious
1                                 words food crapilicious
2                                                   white
3                        classy whore red velvet cupcakes
4       meh p thanks heads concerned another angry dud...
                              ...                        
2951    u tell ppl ok someone ridiculed ur race cuz so...
2952    right battle dumb fuck cuz whites see nigger e...
2953                    mad rt fuck obama dumb ass nigger
2954    fuck dumb nigger would never stand fuckin show...
2955    pussy ass bitch dumb muthafuker like fuck u ni...
Name: instruction, Length: 2956, dtype: object

In [ ]:
#Split in train, test and validation
from sklearn.model_selection import train_test_split

# Split data into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
#Making x_train and y_train columns
X_train = train_df['instruction']
y_train = train_df['output']

X_test = test_df['instruction']
y_test = test_df['output']

In [ ]:
#Bert Medium Model
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [ ]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('gaunernst/bert-medium-uncased')
model = BertModel.from_pretrained('gaunernst/bert-medium-uncased')

# Tokenize and encode the text data using BERT
def encode_text(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True, truncation=True, padding=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Apply encoding to the train and test data
X_train_encoded = [encode_text(text) for text in X_train]
X_test_encoded = [encode_text(text) for text in X_test]

tokenizer_config.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/530 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# Train Logistic Regression model
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X_train_encoded, y_train)

# Make predictions on train set
logreg_train_preds = logreg_model.predict(X_train_encoded)

# Evaluate Logistic Regression model on train set
logreg_train_scores = {
    "accuracy": accuracy_score(y_train, logreg_train_preds),
    "precision": precision_score(y_train, logreg_train_preds, average='weighted'),
    "recall": recall_score(y_train, logreg_train_preds, average='weighted'),
    "f1": f1_score(y_train, logreg_train_preds, average='weighted')
}
print("Logistic Regression Scores:")
print("Accuracy:", logreg_train_scores["accuracy"])
print("Precision:", logreg_train_scores["precision"])
print("Recall:", logreg_train_scores["recall"])
print("F1-Score:", logreg_train_scores["f1"])
print()

Logistic Regression Scores:
Accuracy: 0.9568527918781726
Precision: 0.9568971487233744
Recall: 0.9568527918781726
F1-Score: 0.9568226336212674



In [ ]:
# Train Logistic Regression model - TestData
logreg_model1 = LogisticRegression(max_iter=1000)
logreg_model1.fit(X_test_encoded, y_test)

# Make predictions on train set
logreg_test_preds = logreg_model1.predict(X_test_encoded)

# Evaluate Logistic Regression model on train set
logreg_test_scores = {
    "accuracy": accuracy_score(y_test, logreg_test_preds),
    "precision": precision_score(y_test, logreg_test_preds, average='weighted'),
    "recall": recall_score(y_test, logreg_test_preds, average='weighted'),
    "f1": f1_score(y_test, logreg_test_preds, average='weighted')
}
print("Logistic Regression Scores:")
print("Accuracy:", logreg_test_scores["accuracy"])
print("Precision:", logreg_test_scores["precision"])
print("Recall:", logreg_test_scores["recall"])
print("F1-Score:", logreg_test_scores["f1"])
print()

Logistic Regression Scores:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0



In [ ]:
# Train SVM model
svm_model = SVC()
svm_model.fit(X_train_encoded, y_train)

# Make predictions on train set
svm_train_preds = svm_model.predict(X_train_encoded)

# Evaluate SVM model on train set
svm_train_scores = {
    "accuracy": accuracy_score(y_train, svm_train_preds),
    "precision": precision_score(y_train, svm_train_preds, average='weighted'),
    "recall": recall_score(y_train, svm_train_preds, average='weighted'),
    "f1": f1_score(y_train, svm_train_preds, average='weighted')
}
print("SVM Scores:")
print("Accuracy:", svm_train_scores["accuracy"])
print("Precision:", svm_train_scores["precision"])
print("Recall:", svm_train_scores["recall"])
print("F1-Score:", svm_train_scores["f1"])
print()

SVM Scores:
Accuracy: 0.8472927241962775
Precision: 0.852420289220249
Recall: 0.8472927241962775
F1-Score: 0.8470794106059624



In [ ]:
# Test SVM model
svm_model1 = SVC()
svm_model1.fit(X_test_encoded, y_test)

# Make predictions on train set
svm_test_preds = svm_model1.predict(X_test_encoded)

# Evaluate SVM model on train set
svm_test_scores = {
    "accuracy": accuracy_score(y_test, svm_test_preds),
    "precision": precision_score(y_test, svm_test_preds, average='weighted'),
    "recall": recall_score(y_test, svm_test_preds, average='weighted'),
    "f1": f1_score(y_test, svm_test_preds, average='weighted')
}
print("SVM Scores:")
print("Accuracy:", svm_test_scores["accuracy"])
print("Precision:", svm_test_scores["precision"])
print("Recall:", svm_test_scores["recall"])
print("F1-Score:", svm_test_scores["f1"])
print()

SVM Scores:
Accuracy: 0.893581081081081
Precision: 0.8955381048183013
Recall: 0.893581081081081
F1-Score: 0.8937051388028547



In [ ]:
# Train Neural Network model
nn_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
nn_model.fit(X_train_encoded, y_train)

# Make predictions on train set
nn_train_preds = nn_model.predict(X_train_encoded)

# Evaluate Neural Network model on train set
nn_train_scores = {
    "accuracy": accuracy_score(y_train, nn_train_preds),
    "precision": precision_score(y_train, nn_train_preds, average='weighted'),
    "recall": recall_score(y_train, nn_train_preds, average='weighted'),
    "f1": f1_score(y_train, nn_train_preds, average='weighted')
}
print("Neural Network Scores:")
print("Accuracy:", nn_train_scores["accuracy"])
print("Precision:", nn_train_scores["precision"])
print("Recall:", nn_train_scores["recall"])
print("F1-Score:", nn_train_scores["f1"])
print()

Neural Network Scores:
Accuracy: 0.9919627749576988
Precision: 0.9922185915044651
Recall: 0.9919627749576988
F1-Score: 0.9919795446379417



In [ ]:
# Test Neural Network model
nn_model1 = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
nn_model1.fit(X_test_encoded, y_test)

# Make predictions on train set
nn_test_preds = nn_model1.predict(X_test_encoded)

# Evaluate Neural Network model on train set
nn_test_scores = {
    "accuracy": accuracy_score(y_test, nn_test_preds),
    "precision": precision_score(y_test, nn_test_preds, average='weighted'),
    "recall": recall_score(y_test, nn_test_preds, average='weighted'),
    "f1": f1_score(y_test, nn_test_preds, average='weighted')
}
print("Neural Network Scores:")
print("Accuracy:", nn_test_scores["accuracy"])
print("Precision:", nn_test_scores["precision"])
print("Recall:", nn_test_scores["recall"])
print("F1-Score:", nn_test_scores["f1"])
print()

Neural Network Scores:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0



In [ ]:
# Convert X_train_encoded to NumPy arrays
X_train_encoded = np.array(X_train_encoded)

# Ensure X_train_encoded and y_train have the same number of samples
assert X_train_encoded.shape[0] == len(y_train), "Number of samples in X_train_encoded and y_train should match"

# Reshape X_train_encoded to match the input shape expected by the LSTM model
X_train_encoded_reshaped = X_train_encoded.reshape((X_train_encoded.shape[0], X_train_encoded.shape[1]))


In [ ]:
print(X_train_encoded_reshaped.shape)


(2364, 512)


In [ ]:
# Define the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(X_train_encoded_reshaped.shape[1], 1)))
lstm_model.add(Dense(1, activation='sigmoid'))

# Compile the model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the LSTM model
lstm_model.fit(X_train_encoded_reshaped, y_train, epochs=10, batch_size=32)

# Make predictions on the train set
lstm_train_preds = (lstm_model.predict(X_train_encoded_reshaped) > 0.5).astype(int).flatten()

# Evaluate the LSTM model on the train set
lstm_train_scores = {
    "accuracy": accuracy_score(y_train, lstm_train_preds),
    "precision": precision_score(y_train, lstm_train_preds, average='weighted'),
    "recall": recall_score(y_train, lstm_train_preds, average='weighted'),
    "f1": f1_score(y_train, lstm_train_preds, average='weighted')
}
print("LSTM Scores:")
print("Accuracy:", lstm_train_scores["accuracy"])
print("Precision:", lstm_train_scores["precision"])
print("Recall:", lstm_train_scores["recall"])
print("F1-Score:", lstm_train_scores["f1"])

Epoch 1/10
74/74 [==============================] - 18s 199ms/step - loss: -19.1046 - accuracy: 0.1472
Epoch 2/10
74/74 [==============================] - 15s 198ms/step - loss: -45.0515 - accuracy: 0.1485
Epoch 3/10
74/74 [==============================] - 15s 199ms/step - loss: -60.0358 - accuracy: 0.1485
Epoch 4/10
74/74 [==============================] - 15s 200ms/step - loss: -73.7192 - accuracy: 0.1485
Epoch 5/10
74/74 [==============================] - 15s 201ms/step - loss: -86.9569 - accuracy: 0.1485
Epoch 6/10
74/74 [==============================] - 16s 214ms/step - loss: -99.9428 - accuracy: 0.1485
Epoch 7/10
74/74 [==============================] - 15s 202ms/step - loss: -112.7536 - accuracy: 0.1485
Epoch 8/10
74/74 [==============================] - 15s 201ms/step - loss: -125.4877 - accuracy: 0.1485
Epoch 9/10
74/74 [==============================] - 15s 199ms/step - loss: -138.1143 - accuracy: 0.1485
Epoch 10/10
74/74 [==============================] - 6s 81ms/step
LSTM

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Convert X_test_encoded to NumPy arrays
X_test_encoded = np.array(X_test_encoded)

# Ensure X_test_encoded and y_test have the same number of samples
assert X_test_encoded.shape[0] == len(y_test), "Number of samples in X_test_encoded and y_test should match"

# Reshape X_test_encoded to match the input shape expected by the LSTM model
X_test_encoded_reshaped = X_test_encoded.reshape((X_test_encoded.shape[0], X_test_encoded.shape[1]))


In [ ]:
# Make predictions on the test set
lstm_test_preds = (lstm_model.predict(X_test_encoded_reshaped) > 0.5).astype(int).flatten()

# Evaluate the LSTM model on the test set
lstm_test_scores = {
    "accuracy": accuracy_score(y_test, lstm_test_preds),
    "precision": precision_score(y_test, lstm_test_preds, average='weighted'),
    "recall": recall_score(y_test, lstm_test_preds, average='weighted'),
    "f1": f1_score(y_test, lstm_test_preds, average='weighted')
}
print("LSTM Scores on Test Data:")
print("Accuracy:", lstm_test_scores["accuracy"])
print("Precision:", lstm_test_scores["precision"])
print("Recall:", lstm_test_scores["recall"])
print("F1-Score:", lstm_test_scores["f1"])

19/19 [==============================] - 2s 97ms/step
LSTM Scores on Test Data:
Accuracy: 0.13851351351351351
Precision: 0.01918599342585829
Recall: 0.13851351351351351
F1-Score: 0.033703584890528514


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
